In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - PaliGemma 2 (Deployment)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/instances">
      <img alt="Workbench logo" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" width="32px"><br> Run in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_hf_paligemma2_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_hf_paligemma2_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook provides a practical introduction to using the PaLiGemma 2 model, a powerful vision-language model developed by Google. We'll demonstrate how to leverage its multimodal capabilities to perform tasks like vision question answering. Consult the [model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/paligemma) for more information.


### Objective

- Deploy PaliGemma 2 to a Vertex AI Endpoint.
- Make predictions to the endpoint including:
  - Answering questions about a given image.

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 3. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus). You can request for quota following the instructions at ["Request a higher quota"](https://cloud.google.com/docs/quota/view-manage#requesting_higher_quota).

# @markdown > | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

# Upgrade Vertex AI SDK.
! pip3 install --upgrade --quiet 'google-cloud-aiplatform==1.93.1'

import importlib
# Import the necessary packages
import os
from typing import Any, Dict, Tuple

from google.cloud import aiplatform

if os.environ.get("VERTEX_PRODUCT") != "COLAB_ENTERPRISE":
    ! pip install --upgrade tensorflow
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

models, endpoints = {}, {}
LABEL = "paligemma2"


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)

! gcloud config set project $PROJECT_ID
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
)

## Deploy Model to a Vertex AI Endpoint

In [ ]:
# @title Deploy

MODEL_NAME = "paligemma2-3b-pt-224"  # @param ["paligemma2-3b-pt-224", "paligemma2-3b-mix-224", "paligemma2-3b-ft-docci-448", "paligemma2-3b-mix-448", "paligemma2-3b-pt-448", "paligemma2-3b-pt-896", "paligemma2-10b-mix-224", "paligemma2-10b-pt-224", "paligemma2-10b-ft-docci-448", "paligemma2-10b-mix-448", "paligemma2-10b-pt-448", "paligemma2-10b-pt-896", "paligemma2-28b-mix-224", "paligemma2-28b-pt-224", "paligemma2-28b-mix-448", "paligemma2-28b-pt-448", "paligemma2-28b-pt-896"]
GCS_PREFIX = "gs://vertex-model-garden-restricted-us/paligemma2"

MODEL_ID = os.path.join(GCS_PREFIX, MODEL_NAME)

PUBLISHER_MODEL_NAME = f"publishers/google/models/paligemma@{MODEL_NAME}"


# @markdown If you want to use other accelerator types not listed above, then check other Vertex AI prediction supported accelerators and regions at https://cloud.google.com/vertex-ai/docs/predictions/configure-compute. You may need to manually set the `machine_type`, `accelerator_type`, and `accelerator_count` in the code by clicking `Show code` first.

if "3b" in MODEL_NAME:
    accelerator_type = "NVIDIA_L4"
    machine_type = "g2-standard-16"
    accelerator_count = 1
elif "10b" in MODEL_NAME:
    accelerator_type = "NVIDIA_TESLA_A100"
    machine_type = "a2-highgpu-1g"
    accelerator_count = 1
elif "28b" in MODEL_NAME:
    accelerator_type = "NVIDIA_H100_80GB"
    machine_type = "a3-highgpu-8g"
    accelerator_count = 8
else:
    raise ValueError(f"Recommended GPU setting not found for: {MODEL_NAME}.")

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=False,
)

# @markdown Set use_dedicated_endpoint to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint). Note that [dedicated endpoint does not support VPC Service Controls](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type), uncheck the box if you are using VPC-SC.
use_dedicated_endpoint = True  # @param {type:"boolean"}

In [ ]:
# @title [Option 1] Deploy with Model Garden SDK

# @markdown Deploy with Gen AI model-centric SDK. This section uploads the prebuilt model to Model Registry and deploys it to a Vertex AI Endpoint. It takes 15 minutes to 1 hour to finish depending on the size of the model. See [use open models with Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/open-models/use-open-models) for documentation on other use cases.
from vertexai import model_garden

model = model_garden.OpenModel(PUBLISHER_MODEL_NAME)
endpoints[LABEL] = model.deploy(
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    use_dedicated_endpoint=use_dedicated_endpoint,
    accept_eula=True,  # Accept the End User License Agreement (EULA) on the model card before deploy. Otherwise, the deployment will be forbidden.
)

endpoint = endpoints[LABEL]

In [ ]:
# @title [Option 2] Deploy with customized configs

# @markdown This section uploads the prebuilt PaliGemma 2 models to Model Registry and deploys it to a Vertex AI Endpoint. It takes approximately 15 minutes to finish.

# @markdown Select the desired resolution and precision of prebuilt model to deploy, leaving the optional `custom_paligemma_model_uri` as is. Higher resolution and precision_type can result in better inference results, but may require additional GPU.

TASK = "paligemma_VQA"

# The pre-built serving docker images.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-one-serve:20250205_0822_RC00"


def deploy_model(
    model_name: str = None,
    model_id: str = None,
    task: str = None,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    serving_port: int = 7080,
    serving_route: str = "/predict",
    serving_docker_uri: str = SERVE_DOCKER_URI,
) -> Tuple[aiplatform.Endpoint, aiplatform.Model]:
    """Deploys a model to a real-time prediction endpoint.

    Args:
        model_name: The base name of the model.
        model_id: The model ID.
        task: The task to perform.
        machine_type: The machine type.
        accelerator_type: The accelerator type.
        accelerator_count: The accelerator count.
        serving_port: The serving port.
        serving_route: The serving route.
        hf_token: HuggingFace token for model access.

    Returns:
        A tuple containing the created endpoint and deployed model objects.
    """

    endpoint = aiplatform.Endpoint.create(
        display_name=common_util.get_job_name_with_datetime(prefix=model_name)
    )
    serving_env = {
        "MODEL_ID": model_id,
        "DEPLOY_SOURCE": "notebook",
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=task,
        serving_container_image_uri=serving_docker_uri,
        serving_container_ports=[serving_port],
        serving_container_predict_route=serving_route,
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name="publishers/google/models/paligemma",
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        sync=False,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_hf_paligemma2_deployment.ipynb",
            "NOTEBOOK_ENVIRONMENT": common_util.get_deploy_source(),
        },
    )
    return endpoint, model


endpoints["paligemma2"], models["paligemma2"] = deploy_model(
    model_name=MODEL_NAME,
    model_id=MODEL_ID,
    task=TASK,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    serving_port=7080,
    serving_route="/predict",
    serving_docker_uri=SERVE_DOCKER_URI,
)

In [ ]:
# @title [Optional] Loading an existing Endpoint
# @markdown If you've already deployed an Endpoint, you can load it by filling in the Endpoint's ID below.
# @markdown You can view deployed Endpoints at [Vertex Online Prediction](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).
endpoint_id = ""  # @param {type: "string"}

if endpoint_id:
    endpoint = aiplatform.Endpoint(
        endpoint_name=endpoint_id,
        project=PROJECT_ID,
        location=REGION,
    )

### Predict

The following sections will use images from [pexels.com](https://www.pexels.com/) for demoing purposes. All the images have the following license: https://www.pexels.com/license/.

Images will be resized to a width of 1000 pixels by default since requests made to a Vertex Endpoint are limited to 1.500MB.

In [ ]:
# @title Visual Question Answering

# @markdown This section uses the deployed PaliGemma model to answer questions about a given image.

# @markdown ![](https://images.pexels.com/photos/1006293/pexels-photo-1006293.jpeg?auto=compress&cs=tinysrgb&w=630&h=375&dpr=2)
image_url = "https://images.pexels.com/photos/1006293/pexels-photo-1006293.jpeg"  # @param {type:"string"}

# @markdown You may leave question prompts empty and they will be ignored.
question_prompt = "What is shown in the picture?"  # @param {type: "string"}

# @markdown The question prompt can be non-English languages.


def vqa_predict(
    endpoint: aiplatform.Endpoint,
    image_url: str,
    text_prompt: str,
    parameters: Dict[str, Any] = None,
) -> str:
    """Predicts the answer to a question about an image using an Endpoint,

       and passes parameters in the payload.

    Args:
        endpoint: The deployed Vertex AI endpoint.
        image_url: URL of the image to ask about.
        text_prompt: The text prompt question.
        parameters: Additional parameters for the prediction request.

    Returns:
        The predicted answer string or None if no prediction.
    """

    instances = []
    if text_prompt:
        instances.append(
            {
                "text_prompt": text_prompt,
                "image_url": image_url,
            }
        )

    # Construct the prediction payload
    payload = {"instances": instances}
    if parameters:
        payload["parameters"] = parameters

    response = endpoint.predict(instances=instances, parameters=parameters)
    answer = None
    if response.predictions:
        answer = response.predictions[0]["text"].split("\n")[1]
    return answer


# Using max_new_tokens along with other parameters
parameters_with_tokens = {"max_new_tokens": 50}
predictions_with_tokens = vqa_predict(
    endpoint=endpoints["paligemma2"],
    image_url=image_url,
    text_prompt=question_prompt,
    parameters=parameters_with_tokens,
)

print(f"Prediction Response: {predictions_with_tokens}")
# @markdown Click "Show Code" to see more details.

## Clean up resources

In [ ]:
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME